In [1]:
from numcosmo_py import Ncm, Nc, GObject

import numpy as np
import math
import sys
from astropy.io import fits
from astropy.table import Table
import matplotlib.pyplot as plt

from richness_mass_calib import create_richness_mass_calib

In [2]:
# #Settings
Ncm.cfg_init()
Ncm.cfg_set_log_handler(lambda msg: sys.stdout.write(msg) and sys.stdout.flush())

In [3]:
# Data
DC2_halos_m200c = fits.open(
    "/global/cfs/projectdirs/lsst/groups/CL/cosmoDC2_v1.1.4/extragal/full/halos/halos_m200c_13.0.fits"
)
dt_halos = Table(DC2_halos_m200c[1].data)
# dt_halos['redshift_true', 'mass_fof', 'm200c', 'richness']

In [4]:
rmdata = create_richness_mass_calib(dt_halos)
ascaso = Nc.ClusterMassAscaso()
lnrich_ext = Nc.ClusterMassLnrichExt()

mset = Ncm.MSet()
mset.set(ascaso)

rmdata.m2lnL_val(mset)

13028468.25742934

In [10]:
dset = Ncm.Dataset.new()
dset.append_data(rmdata)

lh = Ncm.Likelihood.new(dset)
mset.param_set_all_ftype(Ncm.ParamType.FREE)

mset.prepare_fparam_map()


fit = Ncm.Fit.factory(
    Ncm.FitType.NLOPT, "ln-neldermead", lh, mset, Ncm.FitGradType.NUMDIFF_CENTRAL
)

fit.run_restart(Ncm.FitRunMsgs.SIMPLE, 1.0e-3, 0.0, None, None)

#----------------------------------------------------------------------------------
# Model fitting. Interating using:
#  - solver:            NLOpt:ln-neldermead
#  - differentiation:   Numerical differentiantion (central)
#...............................................................................................................................
#  Minimum found with precision: |df|/f =  1.00000e-08 and |dx| =  1.00000e-05
#  Elapsed time: 00 days, 00:00:56.4792180
#  iteration            [001186]
#  function evaluations [001188]
#  gradient evaluations [000000]
#  degrees of freedom   [454468]
#  m2lnL     =     406637.332673003 (     406637.33 )
#  Fit parameters:
#     4.23655529934067       0.519979662950475      0.0793878227364521     0.658092946546747      0.0684856750056667     0.150531992866717      5.95880329749316      
#----------------------------------------------------------------------------------
# Restarting:              yes
#  - absolute improvement: 12621830.92

True

In [11]:
fit.log_info()

#----------------------------------------------------------------------------------
# Data used:
#   - NcDataClusterMassRich
#----------------------------------------------------------------------------------
# Model[07000]:
#   - NcClusterMass : Ascaso Ln-normal richness distribution
#----------------------------------------------------------------------------------
# Model parameters
#   -    mup0[00]:  4.45399049919475    [FREE]
#   -    mup1[01]:  0.688640525628584   [FREE]
#   -    mup2[02]:  1.23735391045664    [FREE]
#   - sigmap0[03]:  0.227185566939725   [FREE]
#   - sigmap1[04]: -0.0514021009415301  [FREE]
#   - sigmap2[05]: -0.125441575642385   [FREE]
#   -     cut[06]:  7.36943460201409    [FREE]


In [ ]:
nwalkers = 500

Ncm.func_eval_set_max_threads(2)
Ncm.func_eval_log_pool_stats()

init_sampler = Ncm.MSetTransKernGauss.new(0)
init_sampler.set_mset(mset)
init_sampler.set_prior_from_mset()
init_sampler.set_cov_from_rescale(1.0)

apes = Ncm.FitESMCMCWalkerAPES.new(nwalkers, mset.fparams_len())

esmcmc = Ncm.FitESMCMC.new(fit, nwalkers, init_sampler, apes, Ncm.FitRunMsgs.FULL)
esmcmc.set_nthreads(2)
esmcmc.set_data_file("ascaso1.fits")

esmcmc.start_run()
esmcmc.run(1000)  
esmcmc.end_run()

esmcmc.mean_covar()
fit.log_covar()

# NcmThreadPool:Unused:      0
# NcmThreadPool:Max Unused:  2
# NcmThreadPool:Running:     0
# NcmThreadPool:Unprocessed: 0
# NcmThreadPool:Unused:      2
#----------------------------------------------------------------------------------
# NcmFitESMCMC: Starting Ensemble Sampler Markov Chain Monte Carlo.
#   Number of walkers: 0500.
#   Number of threads: 0002.
#   Using MPI:         no - use MPI enabled but no slaves available.
#----------------------------------------------------------------------------------
# Data used:
#   - NcDataClusterMassRich
#----------------------------------------------------------------------------------
# NcmFitESMCMC: Model set:
#----------------------------------------------------------------------------------
# Model[07000]:
#   - NcClusterMass : Ascaso Ln-normal richness distribution
#----------------------------------------------------------------------------------
# Model parameters
#   -    mup0[00]:  4.45399049919475    [FREE]
#   -    mup1[01]: 

In [ ]:
esmcmc.mean_covar()
fit.log_covar()

In [ ]:
fit.log_covar()

In [ ]:
fit.log_info()